# 🫀 ECG异常检测模型训练与评估

本notebook包含四种深度学习模型的训练和评估：
- CNN1D
- LSTM
- ResNet1D
- Hybrid CNN-LSTM

每个模型都有独立的训练代码块，最后进行综合评估和可视化对比。

## 📦 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import time
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
from src.comparison_models import create_comparison_model
from src.data_loader import PTBDataLoader
from src.model_comparison import ModelComparison

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用设备: {device}')

## 📊 数据加载和预处理

In [ ]:
# 加载数据
print("加载ECG数据...")

# 加载预处理后的数据
X_train = np.load('data/processed/X_train.npy')
X_val = np.load('data/processed/X_val.npy')
X_test = np.load('data/processed/X_test.npy')
y_train = np.load('data/processed/y_train.npy')
y_val = np.load('data/processed/y_val.npy')
y_test = np.load('data/processed/y_test.npy')

print(f'训练集形状: {X_train.shape}, 标签: {y_train.shape}')
print(f'验证集形状: {X_val.shape}, 标签: {y_val.shape}')
print(f'测试集形状: {X_test.shape}, 标签: {y_test.shape}')

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(X_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
y_val_tensor = torch.LongTensor(y_val).to(device)
y_test_tensor = torch.LongTensor(y_test).to(device)

print('数据加载完成！')

## 🔧 训练函数定义

In [ ]:
def train_model(model, model_name, X_train, y_train, X_val, y_val, epochs=50, lr=0.001, batch_size=32):
    """训练模型的通用函数"""
    print(f'\n开始训练 {model_name} 模型...')
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    start_time = time.time()
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_correct = 0
        
        # 批量训练
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_correct += (predicted == batch_y).sum().item()
        
        # 验证阶段
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val)
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_correct = (val_predicted == y_val).sum().item()
        
        # 计算准确率
        train_acc = train_correct / len(X_train)
        val_acc = val_correct / len(X_val)
        
        train_losses.append(train_loss / (len(X_train) // batch_size))
        val_losses.append(val_loss.item())
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss/(len(X_train)//batch_size):.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    
    training_time = time.time() - start_time
    print(f'{model_name} 训练完成！训练时间: {training_time:.2f}秒')
    
    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies,
        'training_time': training_time
    }

def evaluate_model(model, model_name, X_test, y_test):
    """评估模型性能"""
    model.eval()
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model(X_test)
        probabilities = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
    
    inference_time = time.time() - start_time
    
    # 转换为numpy数组进行评估
    y_true = y_test.cpu().numpy()
    y_pred = predicted.cpu().numpy()
    y_prob = probabilities[:, 1].cpu().numpy()  # 异常类别的概率
    
    # 计算各种指标
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    
    # 计算参数数量
    total_params = sum(p.numel() for p in model.parameters())
    
    results = {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'AUC Score': auc,
        'Parameters': total_params,
        'Inference Time (s)': inference_time
    }
    
    print(f'\n{model_name} 评估结果:')
    for key, value in results.items():
        if key != 'Model':
            if 'Time' in key or key == 'Parameters':
                print(f'{key}: {value:.0f}' if key == 'Parameters' else f'{key}: {value:.4f}')
            else:
                print(f'{key}: {value:.4f}')
    
    return results

## 🧠 模型1: CNN1D训练

In [ ]:
# 创建CNN1D模型
print('=' * 60)
print('训练 CNN1D 模型')
print('=' * 60)

cnn1d_model = create_comparison_model('cnn1d', input_size=X_train.shape[1])
print(f'CNN1D模型参数数量: {sum(p.numel() for p in cnn1d_model.parameters()):,}')

# 训练CNN1D模型
cnn1d_trained, cnn1d_history = train_model(
    cnn1d_model, 'CNN1D', 
    X_train_tensor, y_train_tensor, 
    X_val_tensor, y_val_tensor,
    epochs=50, lr=0.001, batch_size=32
)

# 保存模型
torch.save(cnn1d_trained.state_dict(), 'models/cnn1d_model.pth')
print('CNN1D模型已保存到 models/cnn1d_model.pth')

## 🔄 模型2: LSTM训练

In [ ]:
# 创建LSTM模型
print('=' * 60)
print('训练 LSTM 模型')
print('=' * 60)

lstm_model = create_comparison_model('lstm', input_size=X_train.shape[1])
print(f'LSTM模型参数数量: {sum(p.numel() for p in lstm_model.parameters()):,}')

# 训练LSTM模型
lstm_trained, lstm_history = train_model(
    lstm_model, 'LSTM', 
    X_train_tensor, y_train_tensor, 
    X_val_tensor, y_val_tensor,
    epochs=50, lr=0.001, batch_size=32
)

# 保存模型
torch.save(lstm_trained.state_dict(), 'models/lstm_model.pth')
print('LSTM模型已保存到 models/lstm_model.pth')

## 🏗️ 模型3: ResNet1D训练

In [ ]:
# 创建ResNet1D模型
print('=' * 60)
print('训练 ResNet1D 模型')
print('=' * 60)

resnet1d_model = create_comparison_model('resnet1d', input_size=X_train.shape[1])
print(f'ResNet1D模型参数数量: {sum(p.numel() for p in resnet1d_model.parameters()):,}')

# 训练ResNet1D模型
resnet1d_trained, resnet1d_history = train_model(
    resnet1d_model, 'ResNet1D', 
    X_train_tensor, y_train_tensor, 
    X_val_tensor, y_val_tensor,
    epochs=50, lr=0.001, batch_size=32
)

# 保存模型
torch.save(resnet1d_trained.state_dict(), 'models/resnet1d_model.pth')
print('ResNet1D模型已保存到 models/resnet1d_model.pth')

## 🔀 模型4: Hybrid CNN-LSTM训练

In [ ]:
# 创建Hybrid CNN-LSTM模型
print('=' * 60)
print('训练 Hybrid CNN-LSTM 模型')
print('=' * 60)

hybrid_model = create_comparison_model('hybrid_cnn_lstm', input_size=X_train.shape[1])
print(f'Hybrid CNN-LSTM模型参数数量: {sum(p.numel() for p in hybrid_model.parameters()):,}')

# 训练Hybrid CNN-LSTM模型
hybrid_trained, hybrid_history = train_model(
    hybrid_model, 'Hybrid CNN-LSTM', 
    X_train_tensor, y_train_tensor, 
    X_val_tensor, y_val_tensor,
    epochs=50, lr=0.001, batch_size=32
)

# 保存模型
torch.save(hybrid_trained.state_dict(), 'models/hybrid_cnn_lstm_model.pth')
print('Hybrid CNN-LSTM模型已保存到 models/hybrid_cnn_lstm_model.pth')

## 📊 模型评估和对比

In [ ]:
print('=' * 80)
print('模型评估和对比')
print('=' * 80)

# 评估所有模型
models = [
    (cnn1d_trained, 'CNN1D', cnn1d_history),
    (lstm_trained, 'LSTM', lstm_history),
    (resnet1d_trained, 'ResNet1D', resnet1d_history),
    (hybrid_trained, 'Hybrid CNN-LSTM', hybrid_history)
]

results = []

for model, name, history in models:
    result = evaluate_model(model, name, X_test_tensor, y_test_tensor)
    result['Training Time (s)'] = history['training_time']
    results.append(result)

# 创建结果DataFrame
results_df = pd.DataFrame(results)
print('\n' + '=' * 80)
print('所有模型评估结果汇总')
print('=' * 80)
print(results_df.to_string(index=False))

# 保存结果
results_df.to_csv('results/comparison/model_comparison_results.csv', index=False)
print('\n评估结果已保存到 results/comparison/model_comparison_results.csv')

## 📈 训练历史可视化

In [ ]:
# 绘制训练历史
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('模型训练历史对比', fontsize=16, fontweight='bold')

histories = [cnn1d_history, lstm_history, resnet1d_history, hybrid_history]
model_names = ['CNN1D', 'LSTM', 'ResNet1D', 'Hybrid CNN-LSTM']
colors = ['blue', 'red', 'green', 'orange']

# 训练损失
for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
    axes[0, 0].plot(history['train_losses'], label=f'{name}', color=color)
axes[0, 0].set_title('训练损失')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# 验证损失
for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
    axes[0, 1].plot(history['val_losses'], label=f'{name}', color=color)
axes[0, 1].set_title('验证损失')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# 训练准确率
for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
    axes[1, 0].plot(history['train_accuracies'], label=f'{name}', color=color)
axes[1, 0].set_title('训练准确率')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True)

# 验证准确率
for i, (history, name, color) in enumerate(zip(histories, model_names, colors)):
    axes[1, 1].plot(history['val_accuracies'], label=f'{name}', color=color)
axes[1, 1].set_title('验证准确率')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('results/training_history_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('训练历史图表已保存到 results/training_history_comparison.png')

## 🎯 性能对比可视化

In [ ]:
# 性能指标对比
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('模型性能指标对比', fontsize=16, fontweight='bold')

metrics = ['Accuracy', 'F1 Score', 'AUC Score', 'Training Time (s)']
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']

for i, metric in enumerate(metrics):
    ax = axes[i//2, i%2]
    values = results_df[metric].values
    bars = ax.bar(results_df['Model'], values, color=colors[i], alpha=0.8)
    ax.set_title(f'{metric} 对比')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)
    
    # 添加数值标签
    for bar, value in zip(bars, values):
        height = bar.get_height()
        if metric == 'Training Time (s)':
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{value:.1f}s', ha='center', va='bottom', fontweight='bold')
        else:
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('results/performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('性能对比图表已保存到 results/performance_comparison.png')

## 🔍 混淆矩阵可视化

In [ ]:
# 为每个模型生成混淆矩阵
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('模型混淆矩阵对比', fontsize=16, fontweight='bold')

y_true = y_test_tensor.cpu().numpy()

for i, (model, name, _) in enumerate(models):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, 1)
    
    y_pred = predicted.cpu().numpy()
    cm = confusion_matrix(y_true, y_pred)
    
    ax = axes[i//2, i%2]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['正常', '异常'], yticklabels=['正常', '异常'])
    ax.set_title(f'{name} 混淆矩阵')
    ax.set_xlabel('预测标签')
    ax.set_ylabel('真实标签')

plt.tight_layout()
plt.savefig('results/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print('混淆矩阵图表已保存到 results/confusion_matrices.png')

## 🚀 运行完整的可视化评估

In [ ]:
# 运行完整的可视化评估脚本
print('=' * 80)
print('运行完整的可视化评估')
print('=' * 80)

# 运行可视化脚本
exec(open('visualize_all_models.py').read())

print('\n完整的可视化评估已完成！')
print('请查看以下文件夹中的结果:')
print('- results/visualization/ - 所有可视化图表')
print('- view_results.html - 本地HTML展示页面')
print('- EVALUATION_RESULTS.md - 详细评估结果文档')

## 📋 总结和建议

In [ ]:
# 生成总结报告
print('=' * 80)
print('模型训练和评估总结')
print('=' * 80)

# 找出最佳模型
best_accuracy = results_df.loc[results_df['Accuracy'].idxmax()]
best_f1 = results_df.loc[results_df['F1 Score'].idxmax()]
best_auc = results_df.loc[results_df['AUC Score'].idxmax()]
fastest_training = results_df.loc[results_df['Training Time (s)'].idxmin()]
fastest_inference = results_df.loc[results_df['Inference Time (s)'].idxmin()]

print('🏆 性能领先者:')
print(f'最高准确率: {best_accuracy["Model"]} ({best_accuracy["Accuracy"]:.4f})')
print(f'最高F1分数: {best_f1["Model"]} ({best_f1["F1 Score"]:.4f})')
print(f'最高AUC分数: {best_auc["Model"]} ({best_auc["AUC Score"]:.4f})')
print(f'最快训练: {fastest_training["Model"]} ({fastest_training["Training Time (s)"]:.2f}秒)')
print(f'最快推理: {fastest_inference["Model"]} ({fastest_inference["Inference Time (s)"]:.4f}秒)')

print('\n🎯 使用建议:')
print('• 追求最高准确率: 使用 LSTM 模型')
print('• 平衡性能和效率: 使用 CNN1D 模型')
print('• 快速训练需求: 使用 Hybrid CNN-LSTM 模型')
print('• 实时推理应用: 使用 CNN1D 或 Hybrid CNN-LSTM 模型')

print('✅ 所有模型训练和评估已完成！')
print('📊 可视化结果已生成，可通过 view_results.html 查看详细对比')